In [1]:
import stageemi
import stageemi.dev.visu as dev
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import stageemi.dev.decorator_map as dm
import ipywidgets as widg 
import ipyleaflet as ipyl
import datetime as dt
import os 
from ipywidgets import Text, HTML
from ipyleaflet import WidgetControl
import pandas 

In [2]:
file_agat= '../utils/distance_agat.csv'
df_agat = pandas.read_csv(file_agat,sep=',')
df_agat['1'][0:5]

0    3.35
1    0.00
2    1.12
3    2.24
4    5.69
Name: 1, dtype: float64

In [3]:
# Ajout sous la diagonale de la penalite (qui est additive)
penalty_dict={"15-19":5,"11-15":4,"7-11":3,"3-7":2,"0-3":1} # needs to be done in reverse order (high values to low values)
penalty_dict

{'15-19': 5, '11-15': 4, '7-11': 3, '3-7': 2, '0-3': 1}

In [4]:
df_agat.columns[1:31]

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '28', '29', '30', '31', '32'],
      dtype='object')

In [5]:
#df_agat

In [6]:
N=len(df_agat.index)
ind={"0-3":np.nan,"3-7":np.nan,"7-11":np.nan,"11-15":np.nan,"15-19":np.nan}

for i,val in enumerate(df_agat.columns[1:31]): # goes till column 31-1=30
    #i=2 in debug case when one does not start from column 1
    #print(val)

    for j,key in enumerate(penalty_dict):    
#         print(j,key)
        
        if val != '32':
            ind[key] = np.where(np.all(np.array((df_agat[val][i+1::]>=int(key.split('-')[0]),df_agat[val][i+1::]<int(key.split('-')[1]))),axis=0))[0]                  
#             print(ind[key]+i+1)
            if len(ind[key])>0:        
                df_agat[val][ind[key]+i+1]=df_agat[val][ind[key]+i+1]+penalty_dict[key]
        elif val == '32': # special treatment for the last but one column
#             print(df_agat[val][i+1::])
            print(np.all(np.array((list(df_agat[val][i+1::]>=int(key.split('-')[0])),list(df_agat[val][i+1::]<int(key.split('-')[1]))))))
            if np.all(np.array((list(df_agat[val][i+1::]>=int(key.split('-')[0])),list(df_agat[val][i+1::]<int(key.split('-')[1]))))):
                df_agat[val][N-1]=df_agat[val][N-1]+penalty_dict[key]               

<ipython-input-6-15e57b7ddbd1>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agat[val][ind[key]+i+1]=df_agat[val][ind[key]+i+1]+penalty_dict[key]


False
False
True
False
False


<ipython-input-6-15e57b7ddbd1>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agat[val][N-1]=df_agat[val][N-1]+penalty_dict[key]


In [7]:
#df_agat

In [8]:
agat_asym_filename='../utils/distance_agat_asym.csv'
df_agat.to_csv(agat_asym_filename, index = False, header=True) 